# MeowLab

> cat可能是你最常用的命令行工具之一，但是可能很少有人会关注cat的性能问题。
> 
> 但实际上，cat可能比你想的更加高效。在这个lab中，我们会一起来探索cat如此高效的秘密。

## 什么是cat

对于大部分同学来说，这个问题有些太trivial了。但是以防万一，还是在这里介绍一下，cat是一个GNU coreutils中的一个程序，它的作用是连接(con**cat**enate)一系列文件，并将其输出到标准输出流(stdout)中。

> 如果你愿意去找，你会发现这个lab中所有的代码都可以在GNU coreutils中找到，或者你用某个大模型，它可能也能写得不错。
> 但是，除了写代码以外，这个lab中也会包含一些分析和讨论。因此即使你选择直接去抄GNU coreutils中的代码，也不要直接不加思考的复制粘贴。

## 环境要求

* 操作系统：Linux（任何包含GNU coreutils的发行版）
* 编译器：本lab允许使用C/C++或者Rust，选择你喜欢的任何编译器即可。
* Python3.x：本lab一些地方需要你画图，推荐你使用matplotlib。另外，我们使用一个简单的脚本来生成测试文件。

## 在开始之前

这个项目的所有代码需要你自己动手来写，我们只提供了一个用于生成测试文件的脚本。

为了展示比较明显的测试效果，我们会生成一个2GB的测试文件，请确保你的磁盘拥有足够的空间。你可以运行下面这个单元格来查看你的电脑是否有足够的空间。这里我们使用了`df`命令，这个命令的作用是查看某个文件系统所在的磁盘的使用量。

In [3]:
%%bash
df -h /

avail=$(df -h / | awk 'NR==2 {print $4}' | grep -o '[0-9.]*')
unit=$(df -h / | awk 'NR==2 {print $4}' | grep -o '[a-zA-Z]*')
if [[ "$unit" == "M" || "$unit" == "K" ]]; then
    echo "Disk space is low: $avail$unit"
elif [[ "$unit" == "T" ]]; then
    echo "Disk space is sufficient: $avail$unit"
elif [[ "$unit" == "G" ]]; then
    if (( $(echo "$avail < 10" | bc -l) )); then
        echo "Disk space is low: $avail$unit"
    else
        echo "Disk space is sufficient: $avail$unit"
    fi
else
    echo "Unknown unit: $unit"
fi


 /ec  l o c a l h o s t   �Nt0 t   �NtM�n�FO*g\��P0R  W S L 0N A T   !j_N�v  W S L   


Filesystem      Size  Used Avail Use% Mounted on
/dev/sdc       1007G  3.6G  953G   1% /
Disk space is sufficient: 953G


 

你可以使用我们提供的python脚本来生成测试文件，运行下面的单元格。测试文件的生成可能会花费一定的时间。

In [1]:
import random

MB = 1024 * 1024

# A static seed for reproducibility
random.seed(42)

with open("test.txt", "wb") as f:
    for _ in range(2048):
        f.write(random.randbytes(1 * MB)) # 1MB of random data

当前文件夹下面会出现一个大小为2GB的文件`test.txt`。你可以运行下面的单元格来验证。

In [2]:
%%bash
ls -lh test.txt

-rwxrwxrwx 1 lomiland lomiland 2.0G Jun 15 10:07 test.txt


最后，我们的lab会需要使用`hyperfine`来测量程序的运行时间。我们推荐你使用`cargo`进行安装。你可以从[Rust官网](https://www.rust-lang.org/zh-CN/learn/get-started)找到安装脚本。如果你的电脑已经安装好了`cargo`，你可以运行下面的单元格来安装`hyperfine`。

In [8]:
%%bash
cargo install hyperfine

 /ec  l o c a l h o s t   �Nt0 t   �NtM�n�FO*g\��P0R  W S L 0N A T   !j_N�v  W S L   
     Updating crates.io index
  Downloaded hyperfine v1.19.0
  Installing hyperfine v1.19.0
    Updating crates.io index
     Locking 137 packages to latest compatible versions
      Adding colored v2.2.0 (available: v3.0.0)
      Adding indicatif v0.17.4 (available: v0.17.11)
      Adding nix v0.29.0 (available: v0.30.1)
      Adding rand v0.8.5 (available: v0.9.1)
  Downloaded bytecheck_derive v0.6.12
  Downloaded cfg_aliases v0.2.1
  Downloaded autocfg v0.1.8
  Downloaded equivalent v1.0.2
  Downloaded number_prefix v0.4.0
  Downloaded ptr_meta_derive v0.1.4
  Downloaded ptr_meta v0.1.4
  Downloaded statistical v1.0.0
  Downloaded version_check v0.9.5
  Downloaded wyz v0.5.1
  Downloaded bytecheck v0.6.12
  Downloaded tinyvec_macros v0.1.1
  Downloaded anstyle-query v1.1.2
  Downloaded anstyle v1.0.10
  Downloaded clap_lex v0.7.4
  Downloaded anstyle-parse v0.2.6
  Downloaded errno v0.3.12

有了以上的准备工作，我们终于可以开始我们的探索之旅了。

## 任务0: 测量cat的效率

你要做的第一个任务十分简单。学习一下如何使用`hyperfine`，并且使用它来测试GNU coreutils中的cat输出我们生成的测试文件`test.txt`的时间。运行下面的单元格来运行测试。

In [9]:
%%bash
hyperfine --warmup 3 'cat test.txt'

 /ec  l o c a l h o s t   �Nt0 t   �NtM�n�FO*g\��P0R  W S L 0N A T   !j_N�v  W S L   


Benchmark 1: cat test.txt
  Time (mean ± σ):     19.110 s ±  1.131 s    [User: 0.025 s, System: 0.565 s]
  Range (min … max):   16.683 s … 20.382 s    10 runs
 


 

## 任务1: 写一个最朴素的cat

我们现在使用`hyperfine`测量了GNU coreutils中的`cat`的运行效率，但是对于这个运行时间，我们可能并没有什么概念。现在我们将会写一个最朴素的`cat`程序来作为baseline。这个程序需要满足一下要求：
1. 把你的源代码命名为`mycat1.c`或者`mycat1.rs`，并且可执行的二进制文件命名为`mycat1`。
2. 你的程序接受且只接受一个命令行参数，这个参数是你要读取的文件。
3. 你的程序直接使用`read`和`write`系统调用，每次读取并输出一个字符，不使用任何缓冲区。
4. 使用`hpyerfine`测量`mycat1`和`cat`运行时间（`mycat1`的运行时间可能会非常长）
5. 虽然这是一个很简单的程序，但是请仍然保持系统编程的规范。请在你的代码中进行进行所有必要的错误处理。

这个任务不需要在文档中包含任何内容。

In [ ]:
%%bash
hyperfine --warmup 3 --runs 1 './target/mycat1 test.txt'

 /ec  l o c a l h o s t   �Nt0 t   �NtM�n�FO*g\��P0R  W S L 0N A T   !j_N�v  W S L   


Benchmark 1: ./target/mycat1 test.txt


## 任务2: 带有缓冲区的cat

如果你正确完成了上面的任务，你会发现，`mycat1`的性能和系统中的`cat`的性能相差甚远。但是经过了ics课程的学习，你肯定已经想到了至少一个解决方案——使用缓冲区。缓冲区可以有效减少反复进行系统调用的性能消耗。但是，我们要面临的第二个问题是：缓冲区的大小应该如何设置呢？我们接下来将一步一步地探索这个问题。

我们之前肯定也已经讲过不少涉及IO缓冲区的程序了，它们的缓冲区大小都被设置为多少呢？我相信1024, 4096, 8192一定是比较常见的几个数字。经过ics的学习，我相信你们肯定能说出原因了。那么，这个任务，我们将根据这个原理优化我们的`mycat1`。你至少需要完成下面要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat2.c`/`mycat2.rs`，可执行二进制文件命名为`mycat2`。
2. 写一个`io_blocksize`函数用于确定你的缓冲区的大小，在这个任务中，你可以将缓冲区设置成你当前系统中一个内存页的大小。（注意：你不能假设所有系统中内存页的大小都是4K，请你使用一个系统调用或者标准库中的函数来获取内存页的大小，而不要使用一个固定值。不过允许你使用一个固定值，如果获取内存页发生错误，但是这不应该是一个常见的情况）。
3. 使用标准库提供的函数动态分配用于缓冲区的内存。
4. 使用`hpyerfine`测量`mycat2`的运行时间
5. 请保持系统编程的基本规范。

这个任务不需要在文档中包含任何内容。

In [3]:
%%bash
hyperfine --warmup 3 './target/mycat2 test.txt'

Benchmark 1: ./target/mycat2 test.txt
  Time (mean ± σ):     108.510 s ±  6.249 s    [User: 0.344 s, System: 3.483 s]
  Range (min … max):   96.177 s … 119.093 s    10 runs
 


## 任务3: 缓冲区对齐的cat

如果你正确完成了上面这个任务，你会发现，添加了缓冲区的`mycat2`性能提升十分显著。但是我们还可以进一步优化。实际上只是添加了缓冲区并且设置缓冲区的大小为内存页的整数倍并不是没有什么太多的意义，这样的设置只是为了这个一个任务做铺垫的。在这个任务中，我们将尝试将我们的缓冲区对齐到系统的内存页。至于为什么要这么做，请大家在自己的文档中分析一下。你至少需要完成以下要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat3.c`/`mycat3.rs`，可执行二进制文件命名为`mycat3`。
2. 写两个函数`char* align_alloc(size_t size)`和`void align_free(void* ptr)`，它们的作用分别是分配一段内存，长度不小于`size`并且返回一个对齐到内存页起始的指针`ptr`，以及给出一个先前从`align_alloc`返回的指针并释放之前分配的内存。
3. 利用这两个函数修改你的代码，缓冲区的大小仍然设置成一个内存页的大小。
4. 使用`hpyerfine`测量`mycat3`的运行时间
5. 请保持系统编程的基本规范。

这个任务，你需要在文档中回答以下问题：
1. 为什么将缓冲区对齐到系统的内存可能提高性能？你的实验结果支持这个猜想吗？为什么？
2. 为什么我们直接使用`malloc`函数分配的内存不能对齐到内存页，即使我们分配的内存大小已经是内存页大小的整数倍了。
3. 你是怎么在不知道原始的malloc返回的指针的情况下正确释放内存的？

In [4]:
%%bash
hyperfine --warmup 3 './target/mycat3 test.txt'

Benchmark 1: ./target/mycat3 test.txt
  Time (mean ± σ):     111.084 s ±  9.226 s    [User: 0.343 s, System: 3.454 s]
  Range (min … max):   100.913 s … 126.862 s    10 runs
 


1.对齐内存允许DMA直接访问，避免额外的内存复制,减少TLB失效次数，提高虚拟内存效率,优化缓存行使用，减少内存访问延迟,支持SIMD指令（如SSE/AVX）需要内存对齐
2.malloc在分配的内存块前添加元数据（大小信息等）,内存分配器优先考虑内存利用率而非对齐,分配器从内存池中分配，地址取决于池中空闲位置,标准只保证基本对齐（通常是8或16字节），而非页对齐
3.在对齐指针前存储原始指针,分配时：*((void**)(aligned_ptr - sizeof(void*))) = raw_ptr;释放时：void* raw_ptr = *((void**)((char*)ptr - sizeof(void*)));使用原始指针释放：free(raw_ptr);

## 任务4: 设置缓冲区大小为文件系统块大小的整数倍的cat

由于`cat`是涉及文件操作的，所以我们自然不能离开磁盘操作。我们在课内已经学到过，磁盘操作的基本单位是块。并且因为我们操作磁盘是经过了操作系统的一层抽象的，操作系统的文件系统也定义了一个操作文件的基本单位块，这个块的大小和磁盘的块的大小相关，但不总是相同。因此我们操作文件的时候实际接触到的块大小是文件系统的块大小。如果我们每次读取和写入文件的时候都按照文件系统的块大小来进行，也能提升性能。在这个任务中，你至少需要完成以下要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat4.c`/`mycat4.rs`，可执行二进制文件命名为`mycat4`。
2. 修改你的函数`io_blocksize`，让你的缓冲区大小既考虑到内存页大小也考虑到文件系统的块大小。
3. 使用`hyperfine`测量`mycat4`的运行时间。
4. 保持系统编程的基本规范。

> 在完成这项任务的时候你需要注意以下几点：
> 1. 文件系统中的每个文件，块大小不总是相同的。
> 2. 有的文件系统可能会给出虚假的块大小，这种虚假的文件块大小可能根本不是2的整数次幂。

这个任务，你需要在文档中回答以下问题：
1. 为什么在设置缓冲区大小的时候需要考虑到文件系统块的大小的问题？
2. 对于上面提到的两个注意事项你是怎么解决的？

In [1]:
%%bash
hyperfine --warmup 3 './target/mycat4 test.txt'

 /ec  l o c a l h o s t   �Nt0 t   �NtM�n�FO*g\��P0R  W S L 0N A T   !j_N�v  W S L   


Benchmark 1: ./target/mycat4 test.txt
  Time (mean ± σ):     193.073 s ± 20.742 s    [User: 0.439 s, System: 3.694 s]
  Range (min … max):   143.582 s … 215.486 s    10 runs
 


 

1.为了减少磁盘I/O操作，匹配文件系统块大小可以减少部分块读取，提高效率。对齐访问，文件系统块对齐的访问可以减少磁盘控制器的工作量。预读优化，文件系统对块对齐的读取有更好的预读优化。减少内部碎片，避免读取不完整的文件系统块。硬件优化，与物理磁盘扇区大小对齐（通常512B或4K）
2.使用 fstat() 系统调用获取特定文件的块大小，为每个打开的文件单独确定最优缓冲区大小，在打开文件后分配缓冲区，而不是在程序开始时
有效性检查，确保块大小是2的幂，设置合理范围，当获取块大小失败时使用内存页大小

## 任务5: 考虑系统调用开销情况下的cat

如果你正确完成了上面的任务，那么现在你的`cat`已经可以综合考虑内存页大小，内存页对齐和文件系统块大小的因素来设置缓冲区大小了。但是我们会发现，我们自己的`cat`性能仍然不如我们系统中的`cat`。并且如果你尝试过再进一步增大缓冲区的大小，你的`cat`性能还能更高。这是因为我们目前设置的缓冲区大小还不足以掩盖系统调用带来的开销。那么，我们的缓冲区究竟应该设置到什么大小才够呢？其实这个问题，我们仅仅使用理论分析是无法给出答案的，因为答案受到机器的硬件条件，操作系统的涉及等多重不确定因素的影响。但是，我们可以使用实验来解答这个问题。最后，我们还需要做出假设，不考虑上一个任务的注意事项1，也就是我们假设我们文件系统的大部分文件的块大小都一致（你可以使用我们的测试文件的块大小）。因此，设我们在之前的任务中设置的缓冲区大小是buf_size，我们需要通过实验找到一个倍数A，满足以下条件：
1. 当缓冲区大小小于A * buf_size的时候，文件的读写速度显著减小
2. 当缓冲区大小大于A * buf_size的时候，文件的读写速度并不显著提升
最终，我们就可以直接设置我们的`cat`中的缓冲区大小设置成buf_size的固定倍率。在这个任务中，你只少需要完成以下要求：
1. 编写一个实验脚本，尽量排除其他因素的影响，测量只在系统调用开销的影响下，你的系统最合适的缓冲区大小。并且给出这个大小下你的系统的读写速率。
2. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat5.c`/`mycat5.rs`，可执行二进制文件命名为`mycat5`。
3. 利用上面的实验结果，修改你的函数`io_blocksize`。
4. 使用`hyperfine`测量`mycat5`的运行时间。
5. 保持系统编程的基本规范。

> 提示：
> 1. `dd`命令可以用于复制一个文件(也就是读取并写入)，你可以使用命令行参数设置它的缓冲区大小，并且程序终止的时候可以报告平均文件读写速度。
> 2. Linux系统中存在`/dev`文件系统，这个目录下有很多特殊文件，其中有一些对你来说可能有用。`/dev/null`，你向这个文件写入的内容并不真正写入磁盘，并且不会对你的系统造成任何影响。`/dev/zero`，如果你尝试读取这个文件你会得到源源不断的`\0x0`，这个文件也并不真正的从磁盘中读取。

这个任务，你需要在文档中包括以下内容：
1. 解释一下你的实验脚本是怎么设计的。你应该尝试了多种倍率，请将它们的读写速率画成图表包含在文档中。

In [ ]:
# 请在这里填入你的测试脚本
import subprocess
import matplotlib.pyplot as plt
import numpy as np

def run_dd_test(buffer_size):
    # 使用dd命令从/dev/zero读取数据并写入/dev/null
    # 设置count=1000MB来确保测试数据量足够大
    cmd = f"dd if=/dev/zero of=/dev/null bs={buffer_size} count={1000*1024*1024//buffer_size} 2>&1"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    
    # 解析dd的输出获取速度
    output = result.stderr
    for line in output.split('\n'):
        if 'bytes/s' in line:
            speed = float(line.split('(')[1].split(' ')[0])
            return speed
    return None

def main():
    # 基础缓冲区大小（4KB）
    base_size = 4096
    
    # 测试不同的倍数
    multipliers = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
    buffer_sizes = [base_size * m for m in multipliers]
    
    # 存储结果
    speeds = []
    
    # 对每个缓冲区大小进行测试
    for size in buffer_sizes:
        print(f"Testing buffer size: {size/1024:.1f}KB")
        speed = run_dd_test(size)
        if speed:
            speeds.append(speed)
            print(f"Speed: {speed:.2f} MB/s")
        else:
            print("Test failed")
    
    # 绘制结果
    plt.figure(figsize=(10, 6))
    plt.plot([size/1024 for size in buffer_sizes], speeds, 'b-o')
    plt.xlabel('Buffer Size (KB)')
    plt.ylabel('Speed (MB/s)')
    plt.title('Buffer Size vs Read/Write Speed')
    plt.grid(True)
    plt.xscale('log', base=2)
    plt.savefig('buffer_size_test.png')
    plt.close()
    
    # 找出最佳倍数
    best_multiplier = multipliers[np.argmax(speeds)]
    print(f"\n最佳缓冲区大小倍数: {best_multiplier}")
    print(f"对应的缓冲区大小: {base_size * best_multiplier/1024:.1f}KB")
    print(f"最大速度: {max(speeds):.2f} MB/s")

if __name__ == "__main__":
    main()

In [2]:
%%bash
hyperfine --warmup 3 './target/mycat5 test.txt'

 /ec  l o c a l h o s t   �Nt0 t   �NtM�n�FO*g\��P0R  W S L 0N A T   !j_N�v  W S L   


Benchmark 1: ./target/mycat5 test.txt
  Time (mean ± σ):     25.127 s ±  1.993 s    [User: 0.014 s, System: 0.539 s]
  Range (min … max):   21.799 s … 28.276 s    10 runs
 


 

## 任务6: 使用了系统调用`fdadvice`的cat

虽然几乎我们的这个lab都在讨论设置缓冲区大小的问题，但是实际上我们的系统中的`cat`还在其他很多方面做了不少优化。这些优化在多数时候和缓冲区相比都不起决定性作用，但是我们也可以从中学习到不少有趣的技巧。这里我们就只尝试其中一个，使用系统调用`fadvise`。这个系统调用可以提示文件系统我们将会以什么样的模式来读写文件，这样操作系统可以设置合适的readahead窗口为文件的读写做准备。在这个任务中，你需要完成以下要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat6.c`/`mycat6.rs`，可执行二进制文件命名为`mycat6`。
2. 在你的代码中使用`fadvise`进行优化。
3. 使用`hyperfine`测量`mycat6`的运行时间。
4. 保持系统编程的基本规范。

这个任务，你需要在文档中回答以下问题：
1. 你是如何设置`fadvise`的参数的？
2. 对于顺序读写的情况，文件系统可以如何调整readahead？对于随机读写的情况呢？

In [3]:
%%bash
hyperfine --warmup 3 './target/mycat6 test.txt'

 /ec  l o c a l h o s t   �Nt0 t   �NtM�n�FO*g\��P0R  W S L 0N A T   !j_N�v  W S L   


Benchmark 1: ./target/mycat6 test.txt
  Time (mean ± σ):     28.975 s ±  3.627 s    [User: 0.012 s, System: 0.627 s]
  Range (min … max):   25.319 s … 34.554 s    10 runs
 


 

1.使用了三种不同的fadvise提示组合,POSIX_FADV_SEQUENTIAL，提示内核文件将被顺序访问，内核可以增加预读窗口大小，适用于cat这种顺序读取文件的场景。POSIX_FADV_WILLNEED，提示内核文件数据很快会被访问，内核可以提前将数据预读到页面缓存，减少后续读取操作的延迟。POSIX_FADV_NOREUSE,提示内核数据不会被重复使用,内核可以优先回收这些页面缓存,适用于一次性读取的文件操作
2.当文件系统检测到顺序访问模式时，增加预读窗口，聚合I/O请求，预测性预读，优化磁盘调度。对于随机访问模式，减少预读窗口，禁用预读，缓存优化，请求重排序

## 任务7: 总结

经过了上面的所有任务，我们已经成功探索我们系统中最常用的工具`cat`所使用的各种优化。我相信你对涉及系统IO的编程也有了更深刻的理解。现在请你整理汇总上面每个任务你所写的`mycatx`以及系统的`cat`的测量数据，使用一个柱状图来展示。并且请你分析一下你得到的结果：它们符合你的预期吗？为什么？这个结果给你带来了什么启示？

这个任务，你需要在文档中包含以下内容：
1. 你的全部实验结果的柱状图。
2. 你对上述实验结果的分析。

In [1]:
# 这里填入你用于画图的python代码
import matplotlib.pyplot as plt
import numpy as np

# 假设的数据 - 请根据实际测试结果修改这些值
versions = ['mycat1', 'mycat2', 'mycat3', 'mycat4', 'mycat5', 'mycat6']
times = [1805, 108.510, 111.084, 193.073, 25.127, 28.975]  # 单位：秒

# 创建柱状图
plt.figure(figsize=(12, 6))
bars = plt.bar(versions, times)

# 设置图表属性
plt.title('不同版本cat的性能对比')
plt.xlabel('版本')
plt.ylabel('运行时间 (秒)')
plt.xticks(rotation=45)

# 在柱子上方显示具体数值
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.1f}s',
             ha='center', va='bottom')

# 添加网格线
plt.grid(True, axis='y', linestyle='--', alpha=0.7)

# 调整布局
plt.tight_layout()

# 保存图表
plt.savefig('cat_performance_comparison.png')
plt.close()

# 打印性能提升百分比
base_time = times[0]  # mycat1的时间作为基准
print("\n性能提升分析：")
for i, (version, time) in enumerate(zip(versions, times)):
    if i > 0:  # 跳过mycat1
        improvement = (base_time - time) / base_time * 100
        print(f"{version}: 相比mycat1提升了 {improvement:.1f}%")

C:\Users\HONOR\AppData\Local\Temp\ipykernel_25452\2420420592.py:30: UserWarning: Glyph 29256 (\N{CJK UNIFIED IDEOGRAPH-7248}) missing from current font.
  plt.tight_layout()
C:\Users\HONOR\AppData\Local\Temp\ipykernel_25452\2420420592.py:30: UserWarning: Glyph 26412 (\N{CJK UNIFIED IDEOGRAPH-672C}) missing from current font.
  plt.tight_layout()
C:\Users\HONOR\AppData\Local\Temp\ipykernel_25452\2420420592.py:30: UserWarning: Glyph 36816 (\N{CJK UNIFIED IDEOGRAPH-8FD0}) missing from current font.
  plt.tight_layout()
C:\Users\HONOR\AppData\Local\Temp\ipykernel_25452\2420420592.py:30: UserWarning: Glyph 34892 (\N{CJK UNIFIED IDEOGRAPH-884C}) missing from current font.
  plt.tight_layout()
C:\Users\HONOR\AppData\Local\Temp\ipykernel_25452\2420420592.py:30: UserWarning: Glyph 26102 (\N{CJK UNIFIED IDEOGRAPH-65F6}) missing from current font.
  plt.tight_layout()
C:\Users\HONOR\AppData\Local\Temp\ipykernel_25452\2420420592.py:30: UserWarning: Glyph 38388 (\N{CJK UNIFIED IDEOGRAPH-95F4}) miss


性能提升分析：
mycat2: 相比mycat1提升了 94.0%
mycat3: 相比mycat1提升了 93.8%
mycat4: 相比mycat1提升了 89.3%
mycat5: 相比mycat1提升了 98.6%
mycat6: 相比mycat1提升了 98.4%
